## Load Modules

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
os.environ['DISPLAY'] = ':999'
sys.path.append('/home/zachpen87/clearmap/ClearMap2/')

import h5py
import holoviews as hv
import dask.array as da
import scipy.ndimage as ndimage
import skimage
from ClearMap.Environment import * 
import ClearMap.ImageProcessing.H5 as H5img
hv.notebook_extension('bokeh')

## Specify directory information

In [ ]:
channel             = 'fos_790'
directory           = '/home/zachpen87/clearmap/data/SEFL17b/ms_53/' 
hdf5_file           = os.path.join(directory,'data.hdf5')
coords_file         = os.path.join(directory,'coords.npy')

## Import ClearMap2 Workspace

In [ ]:
ws = wsp.Workspace('CellMap', directory=directory);
resources_directory = settings.resources_path

with h5py.File(hdf5_file,'a') as f:
    print('hdf5_file datasets: {x}'.format(x=list(f.keys())))
    print('hdf5_file shape: {x}\n'.format(x=f[channel].shape))
print(ws.info())

coords = np.load(coords_file)
print('slice coordinates: y={y}, x={x}'.format(y=coords[0],x=coords[1]))

## Test parameters

In [ ]:
%output size=80
test_data={}

with h5py.File(os.path.join(directory,'data.hdf5'),'r') as f:
    print(f[channel].shape)
    test_data['raw'] = f[channel][900:930,1500:2000,200:700]
    
    
#smooth with median filter
#as this filter is to remove high frequency, granular noise, ksize of (3,3,3) is often good
test_data['mfilter'] = H5img.array_filter(
    test_data['raw'], 
    filt='median', 
    ksize=(3,3,3))


#estimate background with morphological opening
#kernel size should be at least diameter of larger cells in all dimensions
test_data['bg']  = H5img.array_filter(
    test_data['mfilter'],
    filt='opening', 
    ksize=(5,5,5))


#remove background from median filtered iamge
test_data['bg_rmv'] = test_data['mfilter'] - test_data['bg']


#threshold background subtracted image to prep for subsequent steps
t = 10
print('threshold: {t}'.format(t=t))
test_data['thresh'] = test_data['bg_rmv'].copy()
test_data['thresh'][test_data['bg_rmv']<t] = 0


#calculate distance transform
test_data['dist'] = ndimage.distance_transform_cdt(test_data['thresh'])


#find local max of distance transform
test_data['lmax'] = H5img.array_filter(
    test_data['dist'], 
    filt='local_max',
    ksize=(10,10,10))


#label objects and apply size restriction
test_data['lbls'],mx = ndimage.label(test_data['lmax'], structure=np.ones((3,3,3)))
test_data['lbls'] = H5img.droplbls_arr(test_data['lbls'], min_size = None, max_size=300)


#find centroids
centroids = ndimage.measurements.center_of_mass(
    test_data['bg_rmv'], 
    labels = test_data['lbls'], 
    index = np.unique(test_data['lbls'])[np.where(np.unique(test_data['lbls'])!=0)])
centroids = np.array([list(ctr) for ctr in centroids]).astype('int')
print('objects found: {x}'.format(x=len(centroids)))


#create array representing centers
test_data['ctrs'] = np.zeros(test_data['raw'].shape)
test_data['ctrs'][tuple(centroids.T)]=1
test_data['ctrs'] = H5img.array_filter(test_data['ctrs'], filt='dilation',ksize=(3,3,3))


i=2
p='h'
raw = jvis.gen_hmap(img=test_data['raw'],plane=p,title='raw',inter=i,tools=['hover'],lims=(50,200))
bg = jvis.gen_hmap(img=test_data['bg'],plane=p,title='bg',inter=i,tools=['hover'])
sig = jvis.gen_hmap(img=test_data['bg_rmv'],plane=p,title='bg removed',inter=i,cmap='viridis',tools=['hover'], lims=(0,20))
thresh = jvis.gen_hmap(img=test_data['thresh'],plane=p,title='thresh',inter=i,cmap='viridis',tools=['hover'])
ctrs = jvis.gen_hmap(img=test_data['ctrs'],plane=p,title='centers',inter=i,cmap='Reds',alpha=.6,tools=['hover'])

#(raw + bg + sig + (thresh*ctrs).opts(title='centroids')).cols(2)
(raw + bg + sig + thresh*ctrs).opts(title='centroids').cols(2)

In [ ]:
%output size = 100

#get histogram of holes
freqs, edges = np.histogram(test_data['bg_rmv'], 500)
hist_holes = hv.Histogram((edges, freqs)).opts(
    xlabel = 'Intensity of Background removed image',
    ylabel = 'log(frequency)'
)
hist_holes

# Main Processing Pipeline for Single 3d Image

### Define cropping coordinates

In [ ]:
slc = (slice(None,None), slice(coords[0,0],coords[0,1]), slice(coords[1,0],coords[1,1]))
slc

In [ ]:
chunksize = (500,500,500)

### Smooth image with median filter

In [ ]:
%%time
H5img.hdf5_filter(
    hdf5_file = hdf5_file,
    dset_in = channel,
    dset_out = '_'.join([channel, 'smooth']), 
    filt = 'median', ksize  = (3,3,3),
    chunksize = chunksize,
    slc = slc)

### Estimate background with morphological opening and subtract from smoothed image

In [ ]:
%%time
H5img.hdf5_filter(
    hdf5_file = hdf5_file,
    dset_in = '_'.join([channel, 'smooth']),
    dset_out = '_'.join([channel, 'bg']),
    filt = 'opening', ksize  = (5,5,5),
    chunksize = chunksize)

H5img.hdf5_bgsubtract(
    hdf5_file = hdf5_file,
    dset_img = '_'.join([channel, 'smooth']),
    dset_bg = '_'.join([channel, 'bg']),
    dset_out = '_'.join([channel, 'bg_rmv']),
    chunksize = chunksize)

with h5py.File(hdf5_file,'a') as f:
    print(f.keys())

### Threshold background subtracted image

In [ ]:
%%time
threshold = 10
with h5py.File(hdf5_file,'a') as f:
    image = da.from_array(
            f['_'.join([channel, 'bg_rmv'])],
            chunks = chunksize) 
    image[image<threshold] = 0
    image.to_hdf5(hdf5_file,('/'+'_'.join([channel, 'thresh'])))

### Calculate distance transform

In [ ]:
%%time
H5img.hdf5_filter(
    hdf5_file = hdf5_file,
    dset_in = '_'.join([channel, 'thresh']),
    dset_out = '_'.join([channel, 'dist']),
    filt = 'distance_transform',
    chunksize = chunksize)

### Define local maxima of distance transform

In [ ]:
%%time
H5img.hdf5_filter(
    hdf5_file = hdf5_file,
    dset_in = '_'.join([channel, 'dist']),
    dset_out = '_'.join([channel, 'local_max']),
    filt = 'local_max',
    ksize = (10,10,10),
    chunksize = chunksize)

### Label contiguous local maxima as cells

In [ ]:
%%time
H5img.label(
    hdf5_file = hdf5_file,
    dset_in = '_'.join([channel, 'local_max']),
    dset_out = '_'.join([channel, 'lbls']),
    min_size = None, max_size=500,
    chunk_dimension = 1,
    chunk_size = 400,
    chunk_overlap = 50)

### Find centroids

In [ ]:
%%time
centroids = H5img.find_centroids(
    hdf5_file = hdf5_file,
    dset_lbls = '_'.join([channel, 'lbls']),
    dset_wts = '_'.join([channel, 'bg_rmv']),
    chunk_dimension = 1,
    chunk_size = 400,
    chunk_overlap = 50)
with h5py.File(hdf5_file,'a') as f:
    centroids = centroids.astype('int')
    if '_'.join([channel, 'centroids/original']) in f.keys():
        del f['_'.join([channel, 'centroids/original'])]
    f.create_dataset('_'.join([channel, 'centroids/original']), data=centroids)

### Transform centroids to ABA coordinate plane and define annotation labels

In [ ]:
with h5py.File(hdf5_file,'a') as f:
    centroids = f['_'.join([channel, 'centroids/original'])][:]
    input_shape = f['_'.join([channel, 'bg_rmv'])].shape

    coordinates = res.resample_points(
                      np.flip(centroids,axis=1), sink=None, orientation=None, 
                      source_shape = tuple(np.flip(input_shape)), 
                      sink_shape = io.shape(ws.filename('resampled', postfix=channel.split('_')[1])))

    coordinates = elx.transform_points(
                      coordinates, sink=None, 
                      transform_directory=ws.filename('resampled_to_auto', postfix=channel.split('_')[1]), 
                      binary=True, indices=False);

    coordinates = elx.transform_points(
                      coordinates, sink=None, 
                      transform_directory=ws.filename('auto_to_reference'),
                      binary=True, indices=False);
    
    labels = ano.label_points(
            points =  coordinates,
            annotation_file = '/home/zachpen87/clearmap/AtlasDocs/Horizontal/ABA_25um_annotation.tif',
            key = 'id')
    
    coordinates = np.flip(coordinates, axis=1)
    if '_'.join([channel, 'centroids/transformed']) in f.keys():
        del f['_'.join([channel, 'centroids/transformed'])]
    if '_'.join([channel, 'centroids/labels']) in f.keys():
        del f['_'.join([channel, 'centroids/labels'])]
    f.create_dataset('_'.join([channel, 'centroids/transformed']), data=coordinates)
    f.create_dataset('_'.join([channel, 'centroids/labels']), data=labels) 

## Create heatmap

In [ ]:
ABA_directory = '/home/zachpen87/clearmap/AtlasDocs/Horizontal'
ABA_ref = 'ABA_25um_reference__1_2_3__slice_None_None_None__slice_None_None_None__slice_None_None_None__.tif'
ref = skimage.io.imread(os.path.join(ABA_directory, ABA_ref))

with h5py.File(hdf5_file,'a') as f:
    heatmap = np.zeros(ref.shape)
    coordinates = f['_'.join([channel, 'centroids/transformed'])][:]
    for x in coordinates.astype('int'):
        try:
            heatmap[x[0],x[1],x[2]] += 1
        except:
            pass
    heatmap = H5img.array_filter(heatmap,filt='gaussian',ksize=(2,2,2))
    if '_'.join([channel, 'heatmap']) in f.keys():
        del f['_'.join([channel, 'heatmap'])]
    f.create_dataset('_'.join([channel, 'heatmap']), data = heatmap)

## Display heatmap

In [ ]:
%output size = 150

interval = 5
plane = 'c'

with h5py.File(hdf5_file,'r') as f:
    ABA_directory = '/home/zachpen87/clearmap/AtlasDocs/Horizontal'
    ABA_ref = 'ABA_25um_reference__1_2_3__slice_None_None_None__slice_None_None_None__slice_None_None_None__.tif'
    ref = skimage.io.imread(os.path.join(ABA_directory, ABA_ref))
    ref_i = jvis.gen_hmap(ref,plane=plane,title='Allen Brain Atlas',inter=interval)
    #hmap_i = jvis.gen_hmap(f['_'.join([channel, 'heatmap'])][:],plane=plane,title='Cells',inter=interval,cmap='inferno',lims=(0,.2),alpha=.6,tools=['hover'])

#(ref_i*hmap_i).opts(title='Cells mapped to Allen Brain Atlas')
ref_i

In [ ]:
%output size = 150

interval = 40
plane = 'c'

with h5py.File(hdf5_file,'r') as f:
    ABA_directory = '/home/zachpen87/clearmap/AtlasDocs/Horizontal'
    ABA_ref = 'ABA_25um_reference__1_2_3__slice_None_None_None__slice_None_None_None__slice_None_None_None__.tif'
    ref = skimage.io.imread(os.path.join(ABA_directory, ABA_ref))
    ref_i = jvis.gen_hmap(ref,plane=plane,title='Allen Brain Atlas',inter=interval)
    hmap_i = jvis.gen_hmap(f['_'.join([channel, 'heatmap'])][:],plane=plane,title='Cells',inter=interval,cmap='inferno',lims=(0,.2),alpha=.6,tools=['hover'])

(ref_i*hmap_i).opts(title='Cells mapped to Allen Brain Atlas')

### Delete intermediate files

##### View datasets in file

In [ ]:
with h5py.File(hdf5_file,'a') as f:
    print(f.keys())

##### Select datasets to keep

In [ ]:
kept_dsets = [
    'auto',
    'fos_647',
    'fos_790',
    'fos_790_centroids/transformed',
    'fos_790_centroids/labels',
    'fos_790_heatmap', 
]

with h5py.File(hdf5_file,'a') as f:
    for key in kept_dsets:
        if key not in f.keys():
            print('Warning!!!\n{key} not found in {file}'.format(key=key, file=hdf5_file))

##### Rewrite file, saving only chosen datasets

In [ ]:
temp_file = os.path.join(directory,'data_temp.hdf5')
os.rename(hdf5_file, temp_file)
with h5py.File(temp_file,'r') as tempf, h5py.File(hdf5_file,'w') as f:
    for key in kept_dsets:
        print('writing: {key}'.format(key=key))
        f.create_dataset(key, data=tempf[key][:], compression='lzf')
os.remove(temp_file)